# Sentence similarity

In this notebook we'll run a basic sentence similarity model. This models will be later used for semantic search.

## Installing a sample model

We'll use **sentence-transformers/all-MiniLM-L6-v2**. It's a small and lightweight model that will be good for this particular showcase.

In [1]:
from transformers import AutoModel, AutoTokenizer

model_name = 'sentence-transformers/all-MiniLM-L6-v2'

model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

## Vectorizing sample data

To see if the model is working correctly, we'll try to encode sample data and then try to use another sentence to find the closest passages of text.

In [2]:
from qdrant_client import QdrantClient

client = QdrantClient(host='localhost', port=6333)

Create a vector collection

In [4]:
from qdrant_client.models import Distance, VectorParams

client.recreate_collection(
    collection_name="sample_collection",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

True

Add sample data into the collection

In [5]:
from qdrant_client.models import PointStruct
import torch
import torch.nn.functional as F

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

sentences = [
    "Historia Polski jest bardzo ciekawa. Mieszko I był pierwszym władcą Polski. W roku 966 przyjął chrzest. W ten sposób Polska stała się krajem chrześcijańskim.",
    "Maria Curie-Skłodowska była wybitną polską naukowczynią. Otrzymała dwie Nagrody Nobla. Była pierwszą kobietą, która otrzymała tę nagrodę. Chemia była jej pasją. Odkryła pierwiastki promieniotwórcze.",
    "Trygonometria jest działem matematyki. Zajmuje się ona badaniem zależności między bokami i kątami trójkątów. Trygonometria jest bardzo ważna w nawigacji. Dzięki niej możemy określić położenie statku na morzu.",
]

encoded_sentences = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    model_output = model(**encoded_sentences)

sentence_embeddings = mean_pooling(model_output, encoded_sentences['attention_mask'])
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

client.upsert(
    collection_name="sample_collection",
    points=[
        PointStruct(
            id=idx,
            vector=vector,
            payload={"text": sentences[idx]}
        )
        for idx, vector in enumerate(sentence_embeddings)
    ]
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

Search for similar vectors

In [6]:
query = "Trójkąty są bardzo ważne w matematyce. Wszystkie figury geometryczne są bardzo ciekawe. Matematyka jest fascynująca."

encoded_query = tokenizer(query, padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    model_output = model(**encoded_query)

query_embedding = mean_pooling(model_output, encoded_query['attention_mask'])
query_embedding = F.normalize(query_embedding, p=2, dim=1)

hits = client.search(
    collection_name="sample_collection",
    query_vector=query_embedding[0].tolist(),
    limit=1
)

best_answer = hits[0]

print("Best answer:")
print(best_answer.payload['text'])
print("Similarity:", best_answer.score)

Best answer:
Trygonometria jest działem matematyki. Zajmuje się ona badaniem zależności między bokami i kątami trójkątów. Trygonometria jest bardzo ważna w nawigacji. Dzięki niej możemy określić położenie statku na morzu.
Similarity: 0.61781
